In [2]:
import sys
import os
import numpy as np
import jaydebeapi
import pandas as pd

# 以下是数据把excel整理结果多行插入数据库的例子
url = 'jdbc:oracle:thin:@8.142.180.78:1521:MAEASDB'
driver = 'oracle.jdbc.driver.OracleDriver'
jarFile = 'D:/Tools/JDBC/ojdbc8.jar'
userTo = 'MAEAS'
passwordTo = 'MAEAS'
connTo = jaydebeapi.connect(jclassname=driver,
                          url=url,
                          driver_args=[userTo, passwordTo],
                          jars=jarFile)
# 
# data01 = [['1', 'Ada', '23'], 
#           ['2', 'Black', '19'], 
#           ['3', 'Tim', '30']]
#df=pd.DataFrame(data01,columns=['FREGIONCODE', 'FCATEGORYNUMBER','FCATEGORYNAME'])

# dfn代表新项目，dfh代表老项目,
# dfb代表治疗项目的分类，根据分类形成治疗项目一级和二级的分类编码
dfn = pd.DataFrame(
    pd.read_excel(
        "D:/Works/ProjectDoc/09 上海M/04 构建/集团诊疗项目标准化高值成本匹配到物料编码-叶军2022.5.6(1).xlsx",
        sheet_name='新项目高值耗材匹配-无锡',
        header=0,
        usecols=[0, 1, 2, 6]))
dfb = pd.DataFrame(
    pd.read_excel('D:/C/工作簿1.xlsx',
                  sheet_name='Sheet6',
                  header=0,
                  usecols=None,
                  dtype={2: str}))

#df = pd.DataFrame(pd.concat([dfn, dfh], ignore_index=True).drop_duplicates())
#df.sort_values(by=["项目大类", "项目小类"], ascending=[True, True])
#dfb['大类编码'] = dfb['大类编码'].map(lambda x: str(x))
regon_code = "3202"
#dfb.insert(0, "项目大类编码", dfb["regon_code"].str.cat(dfb["大类编码"], sep=""))
#dfb.insert(2, "项目小类编码", dfb["regon_code"].str.cat(dfb["小类编码"], sep=""))

dfn.insert(0, "regon_code", regon_code)
dfb['大类编码'] = regon_code + dfb['大类编码']
dfb['小类编码'] = regon_code + dfb['小类编码']
# 这里已经merge好了
dfn = pd.merge(dfn, dfb, how='inner').rename(columns={
    '大类编码': '项目大类编码',
    '小类编码': '项目小类编码'
})
dfn.insert(1, '项目大类编码', dfn.pop('项目大类编码'))
dfn.insert(3, '项目小类编码', dfn.pop('项目小类编码'))

# map 用法
# map需要对映射表做去重处理
# dfn.insert(0, '项目大类编码', dfn['项目大类'].map(dfb.set_index('项目大类')['大类编码'].drop_duplicates()))
# dfn.insert(2, '项目小类编码', dfn['项目小类'].map(dfb.set_index('项目小类')['小类编码'].drop_duplicates()))

sf = dfn.drop_duplicates(subset=["项目大类", "治疗项目名称"]).sort_values(
    by=["项目大类", "项目小类", "治疗项目名称"], axis=0)
# 分类排序，把序号形成字符方法1 浮点转成整数
sf.insert(5, '治疗项目编码', sf["项目大类编码"] + '-H' + sf['治疗项目名称'].groupby(sf['项目大类']).rank().astype(int).astype(str).str.zfill(4))

# 分类排序，把序号形成字符方法2 lambda表达式
#sf.insert(5, '治疗项目编码', sf["项目大类编码"] + '-H' + sf['治疗项目名称'].groupby(sf['项目大类']).rank().apply(lambda x:str(x).split(".")[0].zfill(4))

dfn.insert(5, '治疗项目编码',
    dfn['治疗项目名称'].map(sf.set_index('治疗项目名称')['治疗项目编码'].drop_duplicates()))
sf.pop("EAS对应物料编码")
sf=sf.rename(columns={
    'regon_code': 'FREGIONCODE',
    '项目大类编码':'FCATEGORYNUMBER',
    '项目大类':'FCATEGORYNAME',
    '项目小类编码': 'FCATEGORYSUBCODE',
    '项目小类':'FCATEGORYSUBNAME',
    '治疗项目编码':'FPAYITEMNUMBER',
    '治疗项目名称': 'FPAYITEMNAME'
}).sort_values(
    by=["FCATEGORYNUMBER", "FPAYITEMNUMBER"], axis=0)



name='CT_CUS_PAYITEMSCATE_INIT'
columns = sf.columns # 获取要出入数据的列名称
tmp = ''
fields = ''
length_columns = len(columns)
for i in range(length_columns): # 获取要插入的列的数量及插入代码
      tmp = tmp + ':' + str(i+1)
      fields = fields + columns[i]
      if i < length_columns-1:
            tmp += ','
            fields += ','
                # 要执行插入操作的SQL
insert_sql = f'''insert into {name} ({fields}) values({tmp}) '''


insert_data = []
for rows in sf.itertuples(): # 遍历要插入的数据DataFrame
    row = rows[1:] # 获取需要插入的数据。注意：rows[0]是索引。
    insert_data.append(row)
    # cursTo.executemany(insert_sql,insert_data)
    if (rows[0]+1) % 100 == 0 or rows[0] == len(sf)+1: # 每次插入10条。跟上面代码不同的是，我们这里直接使用索引来判断每次插入的数量和是否达到最后一行数据。
        try: 
            cursTo = connTo.cursor()
            cursTo.executemany(insert_sql,insert_data)
            #     #cursTo.commit()
            insert_data = []
            print("执行成功:")
        except Exception as e: 
            #     #sql_exceptions.append((beginrow,endrow, e))
            print('执行过程有问题:',e)
        finally:
            cursTo.close()


执行成功: Pandas(Index=56, FREGIONCODE='3202', FCATEGORYNUMBER='3202-03', FCATEGORYNAME='儿童齿科', FCATEGORYSUBCODE='3202-03-001', FCATEGORYSUBNAME='儿童矫正', FPAYITEMNUMBER='3202-03-H0001', FPAYITEMNAME='MRC肌功能矫治')
